In [568]:
import numpy as np
import pandas as pd
from sound import *
from xgboost import XGBClassifier

df = pd.read_csv('parte_nome_sexo.csv')
df

df = df[df['PrimeiroNomeNoAccents'].notna()]
#df = df[45000:50000]
df = df[:50000]

In [569]:
df['Sounde'] = df.apply(lambda row: soundex(row['PrimeiroNomeNoAccents']), axis=1)

In [570]:
df[df['Masculino'] == df['Feminino']]

,Masculino,Feminino,PrimeiroNomeNoAccents,Sounde


In [571]:
df['SoundexIndice0'] = df[df['Sounde'].notna()].apply(lambda row: ord(row['Sounde'][0:1]), axis=1)
df['SoundexIndice1'] = df[df['Sounde'].notna()].apply(lambda row: int(row['Sounde'][1:2]), axis=1)
df['SoundexIndice2'] = df[df['Sounde'].notna()].apply(lambda row: int(row['Sounde'][2:3]), axis=1)
df['SoundexIndice3'] = df[df['Sounde'].notna()].apply(lambda row: int(row['Sounde'][3:4]), axis=1)

In [572]:
df['Leng'] = df.apply(lambda row: len(row['PrimeiroNomeNoAccents']), axis=1)

In [573]:
for i in range(0, 10):
    df['LetraIndice' + str(i)] = df.apply(lambda row: getascii(row, i), axis=1)
    print('ok {0}'.format(i))

ok 0
ok 1
ok 2
ok 3
ok 4
ok 5
ok 6
ok 7
ok 8
ok 9


In [574]:
def getascii(row, indice):
    c = row['PrimeiroNomeNoAccents'][indice:indice+1]
    if(len(c) == 0):
        return np.nan
    return ord(c)

    

In [575]:
mean = df['LetraIndice1'].mean()
#df_backup = df
s = df[df['LetraIndice1'].isna()]['LetraIndice1']


values = {
    'LetraIndice1': df['LetraIndice1'].mean(), 
    'LetraIndice2': df['LetraIndice2'].mean(), 
    'LetraIndice3': df['LetraIndice3'].mean(), 
    'LetraIndice4': df['LetraIndice4'].mean(), 
    'LetraIndice5': df['LetraIndice5'].mean(), 
    'LetraIndice6': df['LetraIndice6'].mean(), 
    'LetraIndice7': df['LetraIndice7'].mean(), 
    'LetraIndice8': df['LetraIndice8'].mean(), 
    'LetraIndice9': df['LetraIndice9'].mean(), 
}

df = df.fillna(value=values)
df_Mean = pd.DataFrame(values, index=[0])
df_Mean.to_csv("mediaLetras.csv", index=False)

In [576]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df.drop(['Masculino', 'Feminino', 'PrimeiroNomeNoAccents', 'Sounde'], axis=1)
y = df.loc[:, 'Masculino']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=7)

#X
#est = RandomForestClassifier(n_estimators = 40).fit(X, y)
#est.score(X, y)

#est = reg

In [577]:
c = pd.concat([df.loc[:,'PrimeiroNomeNoAccents'], X], axis=1)

In [578]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

parameters =  {
        'learning_rate': [0.02, 0.1],
        'min_child_weight': [0.1, 1],
        'gamma': [0.01, 0.1, 0.5],
        'subsample': [0.8],
        'colsample_bytree': [0.6],
        'max_depth': [5, 8],
        'n_estimators': [300]
        }
#xgb = XGBClassifier()
#clf = GridSearchCV(xgb, parameters, cv=4)
#clf.fit(X, y)


In [579]:
#print(clf.get_params())

#est = clf.best_estimator_
#est
#clf.score(X, y)

In [580]:
est.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.6,
 'gamma': 0.1,
 'learning_rate': 0.02,
 'max_delta_step': 0,
 'max_depth': 8,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 1000,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 0.8}

In [581]:
params1 = {'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.6,
 'gamma': 0.1,
 'learning_rate': 0.02,
 'max_delta_step': 0,
 'max_depth': 8,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 1000,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 0.8}

est = XGBClassifier(**params1)
est.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.1, learning_rate=0.02,
       max_delta_step=0, max_depth=8, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

In [582]:
est.score(X_train, y_train)

0.9676716417910448

In [583]:
est.score(X_test, y_test)

0.9689090909090909

In [584]:
#est.predict_proba(X_test[0:1])
